# It's Week 6!
- time to assemble the dataset
- download data onto CoLab w Drive
- create my baseline model!

# Code From Last Week



### 1. Download the Video

In [2]:
!pip install youtube_dl

     |████████████████████████████████| 1.8MB 2.8MB/s 


In [0]:
# Youtube link download
from __future__ import unicode_literals
import youtube_dl
import os

def download_video(url, dest):
    ydl_opts = {
        'outtmpl': os.path.join(dest, '%(title)s.%(ext)s'),
    }
    
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

### 2. Isolate the Audio

In [4]:
import moviepy.editor as mp

def download_audio(url, dest, subclip_stat=False, clip_start=0, clip_end=0):
    if subclip_stat:
        clip = mp.VideoFileClip(url).subclip(clip_start, clip_end)
    else:
        clip = mp.VideoFileClip(url)
    clip.audio.write_audiofile(dest)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3170304/45929032 bytes (6.9%)7233536/45929032 bytes (15.7%)11403264/45929032 bytes (24.8%)15572992/45929032 bytes (33.9%)19668992/45929032 bytes (42.8%)23781376/45929032 bytes (51.8%)27967488/45929032 bytes (60.9%)32169984/45929032 bytes (70.0%)36028416/45929032 bytes (78.4%)40222720/45929032 bytes (87.6%)44253184/45929032 bytes (96.4%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

### 3. Parsing Dance Clips

In [0]:
import cv2
from PIL import Image

def get_frames(source_url, dest, frames_skipped):
    vid_len_sec = (51 * 60) + 52
    total_frames = vid_len_sec * frames_skipped

    cap = cv2.VideoCapture(source_url)
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
          break
        if count > 150:
          break # for testing

        if count % 1000 == 0:
          print(str(count) + "/" + str(total_frames), "extracted")

        frame = transform_image(frame)
        cv2.imwrite('{:d}.jpg'.format(count), frame)
        count += frames_skipped
        cap.set(1, count)

    cv2.destroyAllWindows()
    print("all", total_frames, "frames extracted!")

### 4. Audio Timestamp Match-Up

In [0]:
# tbd?

### 5. Conversion to Binary Image

In [0]:
def change_contrast(img, level=100):
    factor = (259 * (level + 255)) / (255 * (259 - level))
    
    def contrast(c):
        return 128 + factor * (c - 128)
    
    img = img.point(contrast)
    return img

In [0]:
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
WHITE = (255, 255, 255)
ORANGE = (255, 99, 71)

def remove_colors(img, rgb_remove, rgb_replace):
    width, height = img.size
    
    for x in range(width):
        for y in range(height):
            current_color = img.getpixel((x, y))
            
            if current_color in rgb_remove:
                img.putpixel((x, y), rgb_replace)
#             else:
#                 img.putpixel((x, y), ORANGE)
    
    return img

In [0]:
def remove_blues(img, rgb_replace):
    width, height = img.size
    print(width, height)
    
    for x in range(width):
        for y in range(height):
            current_color = img.getpixel((x, y))
            
            if current_color[2] > 200:
                img.putpixel((x, y), rgb_replace)
            else:
                img.putpixel((x, y), ORANGE) 
    
    return img

### 6. Crop

In [0]:
def my_crop(img):
    width, height = img.size
    
    left = width / 4
    upper = 0
    right = width/1.25
    lower = height - 30
    
    img = img.crop((left, 100, right, lower))
    
    return img

## Final Image Transformation:

In [0]:
def transform_image(orig_img):
  mod = Image.open(orig_img)
  mod = change_contrast(mod, 250)
  mod = remove_colors(mod, BLUES, WHITE)
  mod = remove_blues(mod, WHITE)
  mod = my_crop(mod)
  return mod

# Downloading the Data to Drive!

In [10]:
from google.colab import drive

drive.mount('/content/gdrive')
ROOT = "gdrive/My Drive/Colab Notebooks/spring_2020/data/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [12]:
VID_URL = "https://youtu.be/R7qdgPkPsuQ"
vid_dest = ROOT

root_contents = os.listdir(ROOT)
print("root directory is contains", len(root_contents), "items:", root_contents)

if len(root_contents) == 2:
  download_video(VID_URL, vid_dest)

root directory is contains 4 items: ['frames', 'audio', 'Fortnite All Dances Season 1-11.mp4', 'fortnite_audio.wav']


In [21]:
# Check this value again after the video is done processing
print("root directory is contains", len(root_contents), "items:", root_contents)

root directory is contains 2 items: ['frames', 'audio']


In [0]:
vid_url = vid_dest + "Fortnite All Dances Season 1-11.mp4"
audio_dest = ROOT + "fortnite_audio.wav"
# download_audio(vid_url, audio_dest)

In [24]:
import cv2

frame_dest = ROOT + "frames/test/"
# video runs at 60 fps --> 60 frames skipped == 1 fps pulled
frames_skipped = 60

get_frames(vid_url, frame_dest, frames_skipped)

0/186720 extracted


AttributeError: ignored